# Build Kernel Latency Predictor
Users could refer to the [Guidance](../../docs/builder/build_kernel_latency_predictor.md) for details information.

In [1]:
import warnings
from silence_tensorflow import silence_tensorflow
warnings.filterwarnings('ignore')
silence_tensorflow()

In [2]:
# initialize builder config with workspace
from nn_meter.builder import builder_config
workspace = "/data1/jiahang/working/release2test/"
builder_config.init(workspace) 

In [3]:
# define experiment parameters
kernel_type = "conv-bn-relu"
backend = "debug_backend"
mark = "test"
error_threshold = 0.1

nn-Meter provides APIs from different level to build the latency predictor. The finest grain of APIs initialize the predictor by random sampling, and apply adaptive sampling based on previous data. Here is an example:

In [4]:
from nn_meter.builder import build_initial_predictor_by_data, build_adaptive_predictor_by_data

# initialize kernel latency predictor
predictor, data = build_initial_predictor_by_data(
    kernel_type, backend, init_sample_num=10, mark=mark
)

(nn-Meter) building kernel for conv-bn-relu...
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_priortest_JY51LK.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_priortest_Q07BGY.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_priortest_454622.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_priortest_786ZP3.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_priortest_EP5IDQ.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_priortest_Q07MUE.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/workin

In [5]:
# use the data for init predictor to build adaptive sampler predictor
# this method could be called iteratively to conduct more iterations of adaptive sampling
predictor, data = build_adaptive_predictor_by_data(
    kernel_type, data, backend, finegrained_sample_num=5
)

(nn-Meter) training data size: 7, test data size: 2
(nn-Meter) rmse: 32.1800; rmspe: 186.0863; error: 0.6318; 5% accuracy: 0.0000; 10% accuracy: 0.0000; 15% accuracy: 0.0000.
(nn-Meter) building kernel for conv-bn-relu...
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu__4UVJ0W.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu__L307NB.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu__HPIT4B.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu__FHKQC9.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu__R0YHJA.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/p

Another method `build_predictor_for_kernel` is provided to integrate initial sampling and adaptive sampling. In this method, both `init_sample_num` and `finegrained_sample_num` should be assigned, and the `iteration` indicates the iteration of adaptive sampling. Initial sampling is regarded as iteration 1, thus `iteration == 2` means one iteration for adaptive sampling.

In [6]:
from nn_meter.builder import build_predictor_for_kernel
predictor, data = build_predictor_for_kernel(
    kernel_type, backend, init_sample_num=10, finegrained_sample_num=5, iteration=2
)

(nn-Meter) building kernel for conv-bn-relu...
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_prior_3NF8BP.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_prior_3BJMW3.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_prior_IMMVWF.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_prior_JVRW54.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_prior_L7HTBV.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor_build/models/conv-bn-relu_prior_FFN418.
(nn-Meter) conv-bn-relu model is generated and saved to /data1/jiahang/working/release2test/predictor

nn-Meter also provided a end-to-end method for users to build a series of general latency predictors, named `nn_meter.builder.build_latency_predictor`. This method will build predictors for all kernels in `<workspace-path>/configs/predictorbuild_config.yaml` according to their corresponding parameters. The parameters includes `INIT_SAMPLE_NUM`, `FINEGRAINED_SAMPLE_NUM`, `ITERATION`, and `ERROR_THRESHOLD`. Here is an example:

In [ ]:
# build latency predictor for kernel
from nn_meter.builder import build_latency_predictor
build_latency_predictor(backend="debug_backend")
